In [19]:
# pip install newspaper3k
# pip install gnews
# pip install gradio
# pip install elevenlabs
# pip install openai
# pip install langchain

In [20]:
import os
import gnews
import gradio as gr
from pydub import AudioSegment
from moviepy.editor import concatenate_audioclips, AudioFileClip
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from elevenlabs import voices, generate, play, save
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

eleven_api_key = "ba5f6b65b92726a8912302888e20fea0"

In [21]:
from newspaper import Config

config = Config()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
config.browser_user_agent = user_agent

In [22]:
google_news = gnews.GNews(period='1d', max_results=10, language='sv')
news_response = google_news.get_news("Sprängning Hässelby")

all_news_text = ''
for i in range(len(news_response)): # Iterate collected news
    # try:
    article = google_news.get_full_article(news_response[i]['url']) # Get full news text for each article
    all_news_text += (article.title + ": " + article.text + "****")
    # except:
    #     print('error')
    #     continue

In [23]:
all_news_text

''

In [24]:
num_words = 100
style = "Explain it to me like I'm five years old"
summary_lang = "English"
chat_model = ChatOpenAI(model_name='gpt-3.5-turbo-16k') # Must have set API key as env var
prompt = "Summarize the following text in {} words. {}. The summary must be in {}. The text contains several news articles, separated by this sign '****'. Here's the text: {}".format(num_words, style, summary_lang, all_news_text) 
summary = chat_model.predict(prompt)

In [25]:
def concatenate_audio_moviepy(audio_clip_paths, output_path):
    """Concatenates several audio files into one audio file using MoviePy
    and save it to `output_path`. Note that extension (mp3, etc.) must be added to `output_path`"""
    clips = [AudioFileClip(c) for c in audio_clip_paths]
    final_clip = concatenate_audioclips(clips)
    final_clip.write_audiofile(output_path)

In [ ]:
# wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    # all_news_text = wikipedia.run(topic)    

### IGNORERA ###
    # if topic[0] == 'E':
    #     return "news_audio/summary_in_0.5_mins_0_READY.wav"
    # elif topic[0] == 'T':
    #     return "news_audio/summary_in_1_mins_0_READY.wav"

In [26]:
def podify(topic, style, num_minutes, voice):

    num_words = num_minutes*135 # Approximately 135 words per minute with TTS voice (refine this)

    audio_voice = voice.split()[0]
    summary_lang = "English"
    if voice.split()[1] == "(SWE)":
        summary_lang = "Swedish"
    if voice.split()[1] == "(IT)":
        summary_lang = "Italian"

    google_news = gnews.GNews(period='7d', max_results=10) # News scraper, hitta alternativ! TASK 1.
    news_response = google_news.get_news(topic) # En lista av artikelobjekt, article.text, article.title

    ### TASK 2 ###
    ### Här borde man använda ChatGPT för att gå fråga om artiklarna är relevanta för the user topic ###
    ### Hämta alla articles.title, fråga om "Is this (title) / (first paragraph) relevant to answer (user topic)"
    ### Behöver inte göra en i taget, skicka en lista av titlar/paragrafer i en request - fråga vilka som är relavantas

    all_news_text = ''
    for i in range(len(news_response)): # Iterate collected news
        try:
            article = google_news.get_full_article(news_response[i]['url']) # Get full news text for each article
            all_news_text += (article.title + ": " + article.text + "****") # Experimentera gärna med separators / inget
        except:
            continue      

    ### TASK 3: Nu har jag bara en jävligt lång kontext-version av GPT, men här vill man köra summeringstekniker av all_news_text
    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI(model_name='gpt-3.5-turbo-16k') # Must have set API key as env var
    prompt = "Summarize the following text in {} words. {}. The summary must be in {}. Here's the text: {}".format(num_words, style, summary_lang, all_news_text) # The text contains several news articles, separated by this sign '****'.
    summary = chat_model.predict(prompt)

    # Connect to elevenlabs, generate audio
    audio = generate(text=summary, voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)
    intro_audio = generate(text='You are listening to an AI-generated podcast on the theme {}, provided to you by PerfectPod AI'.format(topic), voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)

    # TASK 4, gör detta bättre. Kom på sätt att göra detta effektivt. Lite onödigt kanske med två olika genereringar av ljud? Hur kan man lägga in ljudeffekter bra? (addrrar ljudfiler, overlayar ljud)

    # Save audiofile
    if not os.path.exists("news_audio"):
        os.makedirs("news_audio")

    # Handling identical file names
    file_suffix = 0
    while os.path.exists("news_audio/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)):
        file_suffix +=1

    save(audio, "news_audio/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix))
    save(intro_audio, "news_audio/summary_in_{}_mins_{}_INTRO.wav".format(num_minutes, file_suffix)) 

    concatenate_audio_moviepy(["news_audio/summary_in_{}_mins_{}_INTRO.wav".format(num_minutes, file_suffix), "sound_effects/original_transition.wav"], "news_audio/summary_in_{}_mins_{}_INTERMED.wav".format(num_minutes, file_suffix))
    concatenate_audio_moviepy(["news_audio/summary_in_{}_mins_{}_INTERMED.wav".format(num_minutes, file_suffix), "news_audio/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)], "news_audio/summary_in_{}_mins_{}_READY.wav".format(num_minutes, file_suffix))

    return "news_audio/summary_in_{}_mins_{}_READY.wav".format(num_minutes, file_suffix)


    ### TASK 5: Hur får vi långa poddar. Vi vill ha lång output, och vi måste experimentera med detta.
    ### Se hur folk generar hela böcker med ChatGPT, medium. Ex generera templates, generera rekursivt delar. Ha med summeringar av tidigare delar?




In [27]:
demo = gr.Interface(fn=podify, 
                    inputs=[
                            gr.Textbox(label="What do you want your podcast to be about?", placeholder="E.g. 'This week's news in AI'"),
                            gr.Textbox(label="Do you want to customize the style of your podcast?", placeholder="E.g. 'Explain this to me like I'm five years old'"),
                            gr.Slider(minimum=0.5, maximum=5, step=0.5, label="Length", info = "Number of minutes of audio summary"),
                            gr.Dropdown(["Adam (US)", "Antoni (IT)", "Arnold (US)", "Bella (US)", "Domi (US)", "Elli (US)", "Josh (SWE)", "Rachel (US)", "Sam (US)"], label="Voice", info="Narrator voice and language of podcast"),
                            ],
                    outputs= ["audio"]
                    )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://c3f9fc3ebcbb84b7ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
